# Suivi de trajectoire - Démonstration (Resnet18 - TensorRT)

Dans ce notebook, nous allons utiliser le modèle entrainé en situation réelle !

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras

# Création de la classe Camera

J'ai créé une classe Image personnelle afin d'optimiser le pipeline avec OpenCV :

In [ ]:
import traitlets
import threading
import atexit
import numpy as np


class Camera(traitlets.HasTraits):
    type_camera = traitlets.Unicode("CSI")
    capture_device = traitlets.Integer(default_value=0)
    capture_width = traitlets.Integer(default_value=1280)
    capture_height = traitlets.Integer(default_value=720)
    display_width = traitlets.Integer(default_value=640)
    display_height = traitlets.Integer(default_value=480)
    fps = traitlets.Integer(default_value=30)
    flip = traitlets.Integer(default_value=0)
    image = traitlets.Any()
    video_on = traitlets.Bool(default_value=False)
    
    def __init__(self,*args,**kwargs):
        super(Camera, self).__init__(*args, **kwargs)
        self._running = False
        self.image = np.zeros((self.display_height, self.display_width, 3), dtype=np.uint8)
        
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)

        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)
    
    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            image_resized = cv2.resize(image,(int(self.display_width),int(self.display_height)))
            return image_resized
        else:
            return self.image
    
    # ON/OFF de la capture vidéo
    def capture_video(self,run=False):
        if run is True:
            self.video_on = True
        else:
            self.video_on = False
    
    # Lecture d'un flux vidéo
    def _capture_video(self):
        while True:
            if not self._running:
                break
            self.image = self.capture_image()

            
    # Détachement de la caméra
    def release(self):
        self.cap.release()

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))
    
    # Surveillance de la variable "video_on"
    @traitlets.observe('video_on')
    def _on_running(self, change):
        if change['new'] and not change['old']:
            # not running -> running
            self._running = True
            self.thread = threading.Thread(target=self._capture_video)
            self.thread.start()
        elif change['old'] and not change['new']:
            # running -> not running
            self._running = False
            self.thread.join()

Fonction pour initialiser la caméra :

In [ ]:
def InitCamera():
    camera = Camera(type_camera="CSI",capture_device=0,
                capture_width=224,capture_height=224,
                display_width=224,display_height=224,
                fps=30,flip=0)
    return camera

# Pilotage automatique

### Algorithme de pilotage automatique

Les variables utilisées dans le calcul du pilotage automatique sont les suivantes :
- Gain_Vitesse : C'est le gain en vitesse apporté au robot
- Offset_Vitesse : C'est l'offset de vitesse à ajouter au calculateur
- Kp : C'est le gain propotionnel du PID
- Kd : C'est le gain dérivé du PID
- Ki : C'est le gain intégral du PID

### Interface de visualisation

On reprend l'interface précédent et on ajoute le calcul de la commande des moteurs ainsi que les sliders permettant de gérer les différents gains :

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
import ipywidgets

# Création des sliders
Gain_Vitesse_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='Gain vitesse')
Offset_Vitesse_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='Offset vitesse')
Commande_moteur_gauche = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='Mot. gauche',orientation='vertical')
Commande_moteur_droit = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='Mot. droit',orientation='vertical')

In [ ]:
import ipywidgets
import traitlets
from IPython.display import display
from jetbot import bgr8_to_jpeg
import tensorrt as trt
import pycuda.autoinit
import pycuda.driver as cuda

class ThreadLive(threading.Thread):
    def __init__(self, state_widget, preview_widget, Gain_Vitesse_slider, Offset_Vitesse_slider, fichier_engine, camera, robot, PRECISION):
        threading.Thread.__init__(self)

        self.cfx = cuda.Device(0).make_context()
              
        TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE)
        runtime = trt.Runtime(TRT_LOGGER)

        # Chargement du moteur
        print("Chargement du moteur...")
        with open(fichier_engine, 'rb') as f:
            buf = f.read()
            engine = runtime.deserialize_cuda_engine(buf)

        # Création du context
        print("Création du context...")
        context = engine.create_execution_context()

        # Allocation des buffers d'entrée / sortie dans la mémoire GPU
        print("Allocation de la mémoire ...")
        size_input = trt.volume(engine.get_binding_shape(0))* engine.max_batch_size
        size_output = trt.volume(engine.get_binding_shape(1))* engine.max_batch_size
        input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
        output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))
        input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
        output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)    
        bindings = [int(input_device_mem), int(output_device_mem)]        
        print("Modèle initialisé ...")
        
        # Sauvegarde dand les variables internes de la classe
        self.context = context
        self.engine  = engine
        self.input_host_mem = input_host_mem
        self.output_host_mem = output_host_mem
        self.input_device_mem = input_device_mem
        self.output_device_mem = output_device_mem
        self.bindings = bindings
        self.PRECISION = PRECISION
        self.state_widget = state_widget
        self.preview_widget = preview_widget
        self.Gain_Vitesse_slider = Gain_Vitesse_slider
        self.Offset_Vitesse_slider = Offset_Vitesse_slider
        self.camera = camera
        self.robot = robot
        self.nbr_step = 0
        self.erreur_1 = 0.0
        self.Kp = 0.05
        self.Ki = 0.0
        self.Kd = 0.2
        
    def CalculCommandes(self,pos_xy, Kp, Ki, Kd, Gain_Vitesse, Offset_Vitesse):
        # Calcul de l'angle de la cible par rapport à la verticale
        angle = np.arctan2(camera.display_width/2 - pos_xy[0],camera.display_height - pos_xy[1])

        # Calcul de l'erreur
        erreur = 0 - angle

        # Calcul de la commande en sortie du PID
        commande = erreur * (Kp + Ki*((1/(self.nbr_step+1))*(self.erreur_1+erreur)) + Kd*(erreur - self.erreur_1))

        # Ajout de l'offset en vitesse
        commande = commande + Offset_Vitesse

        # Mise à jour des variables
        self.erreur_1 = erreur
        self.nbr_step = self.nbr_step + 1

        # Calcul des commandes moteur gauche et droit
        commande_gauche = max(min(Gain_Vitesse + commande, 1.0), 0.0)
        commande_droite = max(min(Gain_Vitesse - commande, 1.0), 0.0)
        return([commande_gauche,commande_droite])
        
    def run(self):
        context = self.context
        input_host_mem = self.input_host_mem
        output_host_mem = self.output_host_mem
        input_device_mem = self.input_device_mem
        output_device_mem = self.output_device_mem
        bindings = self.bindings
        PRECISION = self.PRECISION
        camera = self.camera
        robot = self.robot
        state_widget = self.state_widget
        preview_widget = self.preview_widget
        Gain_Vitesse_slider = self.Gain_Vitesse_slider
        Offset_Vitesse_slider = self.Offset_Vitesse_slider
       
        while state_widget.value == 'live':
            # Capture de l'image
            image_camera = camera.image                # (224,224,3)
            
            # Traitement de l'image
            image = image_camera     # (224,224,3)
            
            # Prédiction
            image = np.asarray(image).astype(trt.nptype(PRECISION))           # (224,224,3)
            image = np.expand_dims(image,axis=0)                              # (1,224,224,3)
            np.copyto(input_host_mem,image.ravel())
            self.cfx.push()
            cuda.memcpy_htod(input_device_mem, input_host_mem)
            context.execute(batch_size=1,bindings=bindings) 
            cuda.memcpy_dtoh(output_host_mem, output_device_mem)    
            self.cfx.pop()
            xy = np.reshape(output_host_mem,(1,2))                            # (1,2)

            # Calcul des coordonnées en pixel
            xs = np.int(camera.display_width * ((xy[0,0]+1) / 2.0))
            ys = np.int(camera.display_height * ((xy[0,1]+1) / 2.0))

            # Calcul de la commande
            commande = self.CalculCommandes([xs,ys], self.Kp, self.Ki, self.Kd, Gain_Vitesse_slider.value, Offset_Vitesse_slider.value)

            # Tracé de la cible sur l'image
            prediction = image_camera.copy()
            prediction = cv2.circle(prediction, (xs, ys), 8, (255, 0, 0), 3)
            preview_widget.value = bgr8_to_jpeg(prediction)
            
            # Commande du robot
            robot.left_motor.value = commande[0]
            robot.right_motor.value = commande[1]


In [ ]:
print("Initialisation de la caméra...")
# Initialise la caméra
try :
    camera.capture_video(run=False)
    camera.release()
    del camera
except NameError:
    pass
camera = InitCamera()

In [ ]:
def start_live(change):
    if change['new'] == 'live':
        global execute_thread
        execute_thread = ThreadLive(state_widget, preview_widget, Gain_Vitesse_slider, Offset_Vitesse_slider, fichier_engine, camera, robot, trt.float32)
        execute_thread.start()
    else:
        execute_thread.join()

In [ ]:
print("Création des widget...")
# Création du widget de la vidéo
camera_widget = ipywidgets.Image(format="jpeg",width=camera.display_width, height=camera.display_height, value=bgr8_to_jpeg(camera.image))
traitlets.dlink((camera, 'image'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
camera_link = traitlets.dlink((camera, 'image'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# Création du widget de visualisaiton des coordonnées estimées
preview_widget = ipywidgets.Image(format="jpeg",width=camera.display_width, height=camera.display_height, value=bgr8_to_jpeg(camera.image))

# Création du widget "state"
state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')

In [ ]:
fichier_engine = "meilleur_model_colab/meilleur_modele_colab_FP16.engine"
PRECISION = trt.float32

# Lance la caméra
camera.capture_video(run=True)

# Création du lien d'observation du widget state
state_widget.observe(start_live, names='value')

# Affichage de l'interface
commandes_widget = ipywidgets.HBox([Commande_moteur_gauche,Commande_moteur_droit])
data_collection_widget = ipywidgets.HBox([camera_widget,preview_widget,commandes_widget])

display(state_widget,data_collection_widget,Gain_Vitesse_slider,Offset_Vitesse_slider)

In [ ]:
robot.stop()